# Tutorial for sipmod

Run the following cell to download libraries:

In [ ]:
!git clone https://github.com/yuccadrew/sipmod.git

# Import the finite element mesh

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('sipmod'))

import numpy as np
from sipmod import *

# Import mesh and compute basis functions
mesh_prefix = 'sipmod/docs/examples/meshes/mesh_ex01'
mesh = MeshTri.read(mesh_prefix+'.1', scale_factor=1e-6)
mesh.draw().show()

# Compute the basis functions

In [ ]:
basis = CellBasisTri(mesh)
basis

# Define the forms

In [ ]:
# Define bilinear form for assembling LSH of the linear system
@BilinearForm
def laplace(u, v, w):
    c = 1.
    return (
        c * u.grad[0] * v.grad[0] +
        c * u.grad[1] * v.grad[1]
    ) * w.area

# Define linear form for assembling RHS of the linear system
@LinearForm
def nonuniform_load(_, w):
    dist = np.sqrt(w.x ** 2 + w.y ** 2)
    f = np.zeros_like(dist)
    mask = dist > 0
    f[mask] = (np.sin(np.pi / 2 * dist[mask]) / dist[mask] +
               np.cos(np.pi / 2 * dist[mask]) * np.pi / 2) * np.pi / 2
    f[~mask] = np.pi ** 2 / 2
    return (f / 3) * w.area

# Assemble and solve the linear system

In [ ]:
# Define any function for your needs
def solution(x, y):
    return np.cos(np.pi * np.sqrt(x ** 2 + y ** 2) / 2)

# Assemble the system of equations
A = laplace.assemble(basis)
b = nonuniform_load.assemble(basis)

# Incoorprate the Dirichlet boundary condition
boundary_nodes = mesh.boundary_nodes('outer')
D = boundary_nodes['id']
s = solution(boundary_nodes['x'], boundary_nodes['y'])
A, b = set_first_kind_bc(A, b, s, D)

# Call the sparse linear system solver
x = solve(A, b)

# Visualize the solution

In [ ]:
mesh.plot(x, cmap='jet').show()

Check the [gallery](https://sipmod.readthedocs.io/en/latest/) for more examples.